In [1]:
%pwd

u'/home/chekalin/Dev/fast-ai-homework/seedling-classification'

In [2]:
HOME = '/home/chekalin/Dev/fast-ai-homework/seedling-classification'

In [3]:
DATA_HOME = HOME + '/data'

In [4]:
import pandas as pd
import os
from glob import glob
import numpy as np
from shutil import copyfile

# Prepare data

In [ ]:
%cd $DATA_HOME

In [ ]:
%rm -rf test/ train/ valid/ results/ sample/ sample_submission.csv

In [ ]:
!unzip test.zip && unzip train.zip && unzip sample_submission.csv.zip

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

In [123]:
categories = os.listdir(DATA_HOME + '/train')

In [ ]:
for category in categories: 
    %mkdir -p "$DATA_HOME/valid/$category"
    %mkdir -p "$DATA_HOME/sample/valid/$category"
    %mkdir -p "$DATA_HOME/sample/train/$category"

In [ ]:
%mkdir -p "$DATA_HOME/results"
%mkdir -p "$DATA_HOME/sample/results"
%mkdir -p "$DATA_HOME/test/unknown"
%mkdir -p "$DATA_HOME/sample/test/unknown"

In [ ]:
% cd $DATA_HOME/test
!echo *.png | xargs mv -t unknown/

In [ ]:
% cd $DATA_HOME
!tree -d

In [ ]:
%cd $DATA_HOME/train
training_files = glob('*/*.png')

In [ ]:
validation_set_size = int(len(training_files) * .1)

shuffled_training_files = np.random.permutation(training_files).tolist()
for i in range(validation_set_size): 
    random_file = shuffled_training_files.pop()
    os.rename(random_file, DATA_HOME + '/valid/' + random_file)

sample_training_set_size = 1000
for i in range(sample_training_set_size): 
    random_file = shuffled_training_files.pop()
    copyfile(random_file, DATA_HOME + '/sample/train/' + random_file)

sample_validation_set_size = 1000
for i in range(sample_validation_set_size): 
    random_file = shuffled_training_files.pop()
    copyfile(random_file, DATA_HOME + '/sample/valid/' + random_file)

In [ ]:
%cd $DATA_HOME/test
test_files = glob('*/*.png')

In [ ]:
sample_test_set_size = 100
for filename in test_files[:sample_test_set_size]: copyfile(filename, DATA_HOME + '/sample/test/' + filename)

In [ ]:
%cd $DATA_HOME
for dirpath, dirnames, filenames in os.walk(DATA_HOME):
    relative = dirpath[len(os.getcwd()):]
    if (len(filenames) and len(relative)): print "Files in {}: {}".format(relative, len(filenames))

# Create model

In [52]:
import numpy as np
np.set_printoptions(precision=4, linewidth=100)

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.data_utils import get_file

def addConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
        model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
addConvBlock(model, 2, 64)
addConvBlock(model, 2, 128)
addConvBlock(model, 3, 256)
addConvBlock(model, 3, 512)
addConvBlock(model, 3, 512)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_33 (BatchNorm (None, 3, 224, 224)   12          batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 3, 226, 226)   0           batchnormalization_33[0][0]      
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
batchnormalization_34 (BatchNorm (None, 64, 224, 224)  256         convolution2d_27[0][0]           
___________________________________________________________________________________________

batchnormalization_45 (BatchNorm (None, 512, 14, 14)   2048        convolution2d_38[0][0]           
____________________________________________________________________________________________________
zeropadding2d_39 (ZeroPadding2D) (None, 512, 16, 16)   0           batchnormalization_45[0][0]      
____________________________________________________________________________________________________
convolution2d_39 (Convolution2D) (None, 512, 14, 14)   2359808     zeropadding2d_39[0][0]           
____________________________________________________________________________________________________
batchnormalization_46 (BatchNorm (None, 512, 14, 14)   2048        convolution2d_39[0][0]           
____________________________________________________________________________________________________
maxpooling2d_15 (MaxPooling2D)   (None, 512, 7, 7)     0           batchnormalization_46[0][0]      
___________________________________________________________________________________________

# Train model

In [54]:
# path = DATA_HOME + '/sample'
path = DATA_HOME
batch_size = 16

In [55]:
from keras.preprocessing import image

def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, 
                                       target_size=(224,224),
                                       class_mode=class_mode, 
                                       shuffle=shuffle, 
                                       batch_size=batch_size)

In [56]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [57]:
batches = get_batches(path + '/train', batch_size=batch_size, gen=gen)
val_batches = get_batches(path + '/valid', batch_size=batch_size)

Found 4275 images belonging to 12 classes.
Found 475 images belonging to 12 classes.


In [58]:
import time

fitting_start_time = time.time()
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=50, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
model.save_weights('weights-50-epochs.h5')
print "fitting 50 epochs took {} seconds".format(int(time.time() - fitting_start_time))

Epoch 1/50
4275/4275 [==============================] - 278s - loss: 3.6649 - acc: 0.2847 - val_loss: 11.8105 - val_acc: 0.1453
Epoch 2/50
4275/4275 [==============================] - 279s - loss: 2.8511 - acc: 0.4012 - val_loss: 5.5046 - val_acc: 0.2295
Epoch 3/50
4275/4275 [==============================] - 278s - loss: 2.5083 - acc: 0.4601 - val_loss: 2.5994 - val_acc: 0.5432
Epoch 4/50
4275/4275 [==============================] - 279s - loss: 2.2371 - acc: 0.5008 - val_loss: 2.0632 - val_acc: 0.5874
Epoch 5/50
4275/4275 [==============================] - 278s - loss: 2.0938 - acc: 0.5160 - val_loss: 5.0370 - val_acc: 0.2442
Epoch 6/50
4275/4275 [==============================] - 278s - loss: 2.1103 - acc: 0.5120 - val_loss: 6.1353 - val_acc: 0.3032
Epoch 7/50
4275/4275 [==============================] - 278s - loss: 2.0528 - acc: 0.5462 - val_loss: 2.5986 - val_acc: 0.5011
Epoch 8/50
4275/4275 [==============================] - 278s - loss: 1.6508 - acc: 0.5855 - val_loss: 1.7174 -

In [59]:
fitting_start_time = time.time()
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=50, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
model.save_weights('weights-100-epochs.h5')
print "fitting 50 took {} seconds".format(int(time.time() - fitting_start_time))

Epoch 1/50
4275/4275 [==============================] - 279s - loss: 2.7872 - acc: 0.2882 - val_loss: 2.8148 - val_acc: 0.2484
Epoch 2/50
4275/4275 [==============================] - 278s - loss: 2.6926 - acc: 0.3027 - val_loss: 1.9908 - val_acc: 0.3979
Epoch 3/50
4275/4275 [==============================] - 278s - loss: 2.5057 - acc: 0.3448 - val_loss: 1.7440 - val_acc: 0.4021
Epoch 4/50
4275/4275 [==============================] - 278s - loss: 2.4588 - acc: 0.3163 - val_loss: 3.3812 - val_acc: 0.3453
Epoch 5/50
4275/4275 [==============================] - 278s - loss: 2.5225 - acc: 0.3102 - val_loss: 2.5618 - val_acc: 0.2821
Epoch 6/50
4275/4275 [==============================] - 279s - loss: 2.3399 - acc: 0.2559 - val_loss: 1.8523 - val_acc: 0.3684
Epoch 7/50
4275/4275 [==============================] - 278s - loss: 2.2657 - acc: 0.2758 - val_loss: 2.0235 - val_acc: 0.3811
Epoch 8/50
4275/4275 [==============================] - 280s - loss: 2.1721 - acc: 0.2933 - val_loss: 1.8387 - 

In [63]:
model.optimizer.lr=0.00001

In [64]:
model.load_weights('weights-100-epochs.h5')
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=10, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
model.save_weights('weights-110-epochs.h5')

Epoch 1/10
4275/4275 [==============================] - 277s - loss: 1.2509 - acc: 0.5705 - val_loss: 1.4501 - val_acc: 0.6084
Epoch 2/10
4275/4275 [==============================] - 279s - loss: 1.2487 - acc: 0.5761 - val_loss: 1.7338 - val_acc: 0.4568
Epoch 3/10
4275/4275 [==============================] - 277s - loss: 1.2075 - acc: 0.5841 - val_loss: 1.1072 - val_acc: 0.6842
Epoch 4/10
4275/4275 [==============================] - 277s - loss: 1.1531 - acc: 0.5913 - val_loss: 1.1819 - val_acc: 0.6863
Epoch 5/10
4275/4275 [==============================] - 277s - loss: 1.1685 - acc: 0.6012 - val_loss: 1.0892 - val_acc: 0.6968
Epoch 6/10
4275/4275 [==============================] - 277s - loss: 1.1062 - acc: 0.6201 - val_loss: 1.0812 - val_acc: 0.7032
Epoch 7/10
4275/4275 [==============================] - 277s - loss: 1.1024 - acc: 0.6122 - val_loss: 0.9141 - val_acc: 0.7326
Epoch 8/10
4275/4275 [==============================] - 276s - loss: 1.1824 - acc: 0.6047 - val_loss: 1.9564 - 

In [65]:
model.optimizer.lr=0.0001

In [66]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=10, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
model.save_weights('weights-120-epochs.h5')

Epoch 1/10
4275/4275 [==============================] - 275s - loss: 1.0303 - acc: 0.6442 - val_loss: 1.0373 - val_acc: 0.6947
Epoch 2/10
4275/4275 [==============================] - 276s - loss: 1.0539 - acc: 0.6370 - val_loss: 0.9747 - val_acc: 0.6674
Epoch 3/10
4275/4275 [==============================] - 278s - loss: 1.0247 - acc: 0.6494 - val_loss: 0.8441 - val_acc: 0.7032
Epoch 4/10
4275/4275 [==============================] - 277s - loss: 0.9343 - acc: 0.6849 - val_loss: 0.8653 - val_acc: 0.7284
Epoch 5/10
4275/4275 [==============================] - 277s - loss: 0.9919 - acc: 0.6625 - val_loss: 1.0644 - val_acc: 0.7326
Epoch 6/10
4275/4275 [==============================] - 277s - loss: 0.9664 - acc: 0.6685 - val_loss: 1.0918 - val_acc: 0.7642
Epoch 7/10
4275/4275 [==============================] - 276s - loss: 0.9347 - acc: 0.6765 - val_loss: 8.1395 - val_acc: 0.2442
Epoch 8/10
4275/4275 [==============================] - 277s - loss: 1.0525 - acc: 0.6421 - val_loss: 1.3795 - 

In [73]:
model.optimizer.lr=0.00001

In [74]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=100, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
model.save_weights('weights-220-epochs.h5')

Epoch 1/100
4275/4275 [==============================] - 275s - loss: 1.0118 - acc: 0.6664 - val_loss: 1.0108 - val_acc: 0.7137
Epoch 2/100
4275/4275 [==============================] - 278s - loss: 0.9127 - acc: 0.6837 - val_loss: 1.0477 - val_acc: 0.7474
Epoch 3/100
4275/4275 [==============================] - 278s - loss: 0.8954 - acc: 0.6835 - val_loss: 0.9005 - val_acc: 0.7621
Epoch 4/100
4275/4275 [==============================] - 278s - loss: 0.9105 - acc: 0.6896 - val_loss: 0.7694 - val_acc: 0.7705
Epoch 5/100
4275/4275 [==============================] - 278s - loss: 0.8530 - acc: 0.7092 - val_loss: 1.2092 - val_acc: 0.7347
Epoch 6/100
4275/4275 [==============================] - 277s - loss: 0.8372 - acc: 0.7113 - val_loss: 1.0042 - val_acc: 0.7432
Epoch 7/100
4275/4275 [==============================] - 278s - loss: 0.7944 - acc: 0.7308 - val_loss: 1.0605 - val_acc: 0.7137
Epoch 8/100
4275/4275 [==============================] - 278s - loss: 0.8020 - acc: 0.7310 - val_loss: 0

4275/4275 [==============================] - 276s - loss: 0.4801 - acc: 0.8346 - val_loss: 0.4604 - val_acc: 0.8968
Epoch 65/100
4275/4275 [==============================] - 276s - loss: 0.4647 - acc: 0.8337 - val_loss: 0.3989 - val_acc: 0.8863
Epoch 66/100
4275/4275 [==============================] - 275s - loss: 0.4233 - acc: 0.8505 - val_loss: 0.4860 - val_acc: 0.8737
Epoch 67/100
4275/4275 [==============================] - 276s - loss: 0.4464 - acc: 0.8414 - val_loss: 0.5746 - val_acc: 0.8547
Epoch 68/100
4275/4275 [==============================] - 275s - loss: 0.4606 - acc: 0.8384 - val_loss: 0.7716 - val_acc: 0.8632
Epoch 69/100
4275/4275 [==============================] - 275s - loss: 0.4414 - acc: 0.8470 - val_loss: 0.6584 - val_acc: 0.8442
Epoch 70/100
4275/4275 [==============================] - 275s - loss: 0.5339 - acc: 0.8133 - val_loss: 0.6580 - val_acc: 0.8758
Epoch 71/100
4275/4275 [==============================] - 276s - loss: 0.4248 - acc: 0.8508 - val_loss: 0.6180

In [113]:
val_batches = get_batches(path + '/valid', batch_size=batch_size, shuffle=False)

Found 475 images belonging to 12 classes.


In [114]:
predictions = model.predict_generator(val_batches, val_batches.nb_sample)

In [115]:
predicted_classes = np.argmax(predictions, axis = 1)
actual_classes = val_batches.classes

In [116]:
from sklearn.metrics import precision_score, log_loss, recall_score, f1_score

precision = precision_score(actual_classes, predicted_classes, average='micro');
recall = recall_score(actual_classes, predicted_classes, average='micro')
f1 = (2 * precision * recall) / (precision + recall)
f1_calculated = f1_score(actual_classes, predicted_classes, average='micro')

print 'Precision score: {:.4f}%'.format(precision)
print 'Recall score: {:.4f}%'.format(recall)
print 'F1: {:.4f}%'.format(f1)
print 'F1 calculated: {:.4f}%'.format(f1_calculated)

Precision score: 0.8800%
Recall score: 0.8800%
F1: 0.8800%
F1 calculated: 0.8800%


In [117]:
idxs = np.where(actual_classes == predicted_classes)[0]
print '{} correct predictions found'.format(len(idxs))


418 correct predictions found


# Run prediction and prepare submission

In [118]:
test_batches = get_batches(path + '/test', batch_size=batch_size, shuffle=False)

Found 794 images belonging to 1 classes.


In [119]:
predictions = model.predict_generator(test_batches, test_batches.nb_sample)

12

In [127]:
import csv

results_file = path + '/results/submission.csv'
file = open(results_file, "wb")
writer = csv.writer(file, delimiter=',', quoting=csv.QUOTE_NONE)
header = ['file','species']
writer.writerow(header)

predicted_classes = np.argmax(predictions, axis = 1)


for filename, prediction in zip(test_batches.filenames, predicted_classes):
    writer.writerow([filename[len('unknown/'):]] + [sorted(categories)[prediction]])

file.close()

In [128]:
from IPython.display import FileLink
%cd $HOME
relative_path_to_results = results_file.replace(HOME, '')[1:]
FileLink(relative_path_to_results)

/home/chekalin/Dev/fast-ai-homework/seedling-classification


/home/chekalin/Dev/fast-ai-homework/seedling-classification/data/results/submission.csv